^fill^README.md^here^

- - -

^fill^theory.md^here^

- - -

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import glob
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

### 2. Assign values for the calculation's run parameters

#### 2.0 Set the calculation's working directory

In [2]:
calc_name = 'LAMMPS_ELASTIC'

# Check current working directory
cwd_name = os.path.basename(os.getcwd())

# Change working directory if needed
if cwd_name != calc_name:
    if not os.path.isdir(calc_name):
        os.mkdir(calc_name)
    os.chdir(calc_name)

#### 2.1 Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use (required).

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

- __lib_directory__ defines the relative path to the iprPy library. This makes it easier to define paths to reference records later.

In [3]:
lammps_command = 'lmp_serial'
mpi_command = None
lib_directory = '../../../library'

#### 2.2 Specify the prototype system

- __system__ is an atomman.System to perform the scan on (required). 

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to use for ucell. 

- __prototype_path__ gives the path to the crystal_prototype reference record to use.

- __box_parameters__ defines the initial guess box parameters to scale ucell and system by.

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

In [4]:
prototype_name = 'A1--Cu--fcc'
box_parameters = [3.5, 3.5, 3.5]
sizemults = [3,3,3]

# Define prototype_path using lib_directory and prototype_name
prototype_path = os.path.abspath(os.path.join(lib_directory, 
                                              'crystal_prototype', 
                                              prototype_name+'.json'))

# Create ucell by loading prototype record
ucell = am.load('system_model', prototype_path)[0]
print('# of atoms in ucell =', ucell.natoms)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], 
              b=box_parameters[1], 
              c=box_parameters[2], scale=True)

# Generate system by supersizing ucell
system = am.supersize(ucell, *sizemults)
print('# of atoms in system =', system.natoms)

# of atoms in ucell = 4
# of atoms in system = 108


#### 2.3 Specify the potenital and elemental symbols

- __potential__ is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use (required).

- __symbols__ is a list of the elemental model symbols of potential to associate with the unique atom types of system (required).

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for potential. 

- __potential_path__ gives the path to the potential_LAMMPS reference record to use.

- __potential_dir_path__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file)



In [5]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
symbols = ['Ni']

# Define potential_path and potential_dir_path using lib_directory and potential_name
potential_dir_path = os.path.abspath(os.path.join(lib_directory, 
                                                  'potential_LAMMPS',
                                                  potential_name))
potential_path = potential_dir_path + '.json'

# Create potential by loading LAMMPS-potential record
with open(potential_path) as f:
    potential = lmp.Potential(f, potential_dir_path)
print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.4 Specify calculation-specific run parameters

- __strainrange__ specifies the $\Delta \epsilon$ strain range to use in estimating $C_{ij}$.

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units.

- __pressure_unit__ is the unit of pressure to use for reporting the elastic constants. 

- __convergence_tol__ is the relative tolerance to use in identifying if the lattice constants have converged. 

In [6]:
strainrange = 1e-7
energytolerance = 1e-8
forcetolerance = uc.set_in_units(0.0, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')
pressure_unit = 'GPa'
convergence_tol = 1e-10

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 in.elastic

In [7]:
with open('in.elastic', 'w') as f:
    f.write("""^fill^in.elastic^here^
""")

#### 3.2 displace.mod

In [8]:
with open('displace.mod', 'w') as f:
    f.write("""^fill^displace.mod^here^
""")

#### 3.3 init.mod.template

In [9]:
with open('init.mod.template', 'w') as f:
    f.write("""^fill^init.mod.template^here^
""")

#### 3.4 potential.mod.template

In [10]:
with open('potential.mod.template', 'w') as f:
    f.write("""^fill^potential.mod.template^here^
""")

#### 3.5 lammps_ELASTIC_refine()

In [11]:
^fill^calc_LAMMPS_ELASTIC.py(lammps_ELASTIC_refine)^here^

#### 3.6 lammps_ELASTIC()

In [12]:
^fill^calc_LAMMPS_ELASTIC.py(lammps_ELASTIC)^here^

### 4. Run calculation function(s)

In [13]:
results_dict = lammps_ELASTIC_refine(lammps_command, 
                                     system, 
                                     potential,
                                     symbols,
                                     mpi_command = mpi_command,
                                     ucell=ucell,
                                     etol=1e-8,
                                     strainrange = strainrange,
                                     maxiter=1000,
                                     maxeval=10000)

In [14]:
results_dict.keys()

['stress',
 'gamma_lat',
 'a_lat',
 'beta_lat',
 'b_lat',
 'alpha_lat',
 'c_lat',
 'C_elastic',
 'E_coh',
 'system_relaxed']

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display relaxed lattice constants in.
- __energy_unit__ is the unit of energy to display cohesive energies in.
- __pressure_unit__ is the unit of pressure to display Cij and relaxed stress state in.

In [15]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'

#### 5.2 Print Ecoh and lattice constants of relaxed ucell

In [16]:
print('Ecoh =', uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)
print('a =',    uc.get_in_units(results_dict['a_lat'], length_unit), length_unit)
print('b =',    uc.get_in_units(results_dict['b_lat'], length_unit), length_unit)
print('c =',    uc.get_in_units(results_dict['c_lat'], length_unit), length_unit)
print('alpha =', results_dict['alpha_lat'])
print('beta = ', results_dict['beta_lat'])
print('gamma =', results_dict['gamma_lat'])

Ecoh = -4.45 eV
a = 3.51999933333 angstrom
b = 3.51999933333 angstrom
c = 3.51999933333 angstrom
alpha = 90.0
beta =  90.0
gamma = 90.0


#### 5.3 Print Cij

In [17]:
print('Cij ('+pressure_unit+') =')
for Ci in uc.get_in_units(results_dict['C_elastic'].Cij, pressure_unit):
    print('[%9.4f %9.4f %9.4f %9.4f %9.4f %9.4f]' % tuple(Ci))

Cij (GPa) =
[ 247.8624  147.8284  147.8284    0.0000    0.0000    0.0000]
[ 147.8284  247.8624  147.8284    0.0000    0.0000    0.0000]
[ 147.8284  147.8284  247.8624    0.0000    0.0000    0.0000]
[   0.0000    0.0000    0.0000  124.8381    0.0000    0.0000]
[   0.0000    0.0000    0.0000    0.0000  124.8381    0.0000]
[   0.0000    0.0000    0.0000    0.0000    0.0000  124.8381]


#### 5.4 Check stress state of relaxed system

In [18]:
print('stress ('+pressure_unit+') =')
for Si in uc.get_in_units(results_dict['stress'], pressure_unit):
    print('[%9.4f %9.4f %9.4f]' % tuple(Si))

stress (GPa) =
[   0.0000    0.0000    0.0000]
[   0.0000    0.0000    0.0000]
[   0.0000    0.0000    0.0000]


#### 5.5 Show relaxed atomic configuration

In [19]:
print(results_dict['system_relaxed'])

avect =  [10.560,  0.000,  0.000]
bvect =  [ 0.000, 10.560,  0.000]
cvect =  [ 0.000,  0.000, 10.560]
origin = [-0.030, -0.030, -0.030]
natoms = 108
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |  -0.030 |  -0.030 |  -0.030
      1 |       1 |  -0.030 |   1.730 |   1.730
      2 |       1 |   1.730 |  -0.030 |   1.730
      3 |       1 |   1.730 |   1.730 |  -0.030
      4 |       1 |   3.490 |  -0.030 |  -0.030
      5 |       1 |   3.490 |   1.730 |   1.730
      6 |       1 |   5.250 |  -0.030 |   1.730
      7 |       1 |   5.250 |   1.730 |  -0.030
      8 |       1 |   7.010 |  -0.030 |  -0.030
      9 |       1 |   7.010 |   1.730 |   1.730
     10 |       1 |   8.770 |  -0.030 |   1.730
     11 |       1 |   8.770 |   1.730 |  -0.030
     12 |       1 |  -0.030 |   3.490 |  -0.030
     13 |       1 |  -0.030 |   5.250 |   1.730
     14 |       1 |   1.730 |   3.490 |   1.730
     15 |       1 |   1.730 |   5.250 |  -0.030
     16 |       1 |   3